In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.ndimage import zoom
from concurrent.futures import ThreadPoolExecutor, as_completed
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Define paths based on your directory structure
main_dir = "/content/drive/My Drive/DATASET_MRNET/MRNet-v1.0"
train_path = os.path.join(main_dir, "train")
valid_path = os.path.join(main_dir, "valid")

In [ ]:
# Load labels from CSV files
train_abnormal = pd.read_csv(os.path.join(main_dir, "train-abnormal.csv"), header=None, index_col=0).squeeze("columns").to_dict()
train_acl = pd.read_csv(os.path.join(main_dir, "train-acl.csv"), header=None, index_col=0).squeeze("columns").to_dict()
train_meniscus = pd.read_csv(os.path.join(main_dir, "train-meniscus.csv"), header=None, index_col=0).squeeze("columns").to_dict()

valid_abnormal = pd.read_csv(os.path.join(main_dir, "valid-abnormal.csv"), header=None, index_col=0).squeeze("columns").to_dict()
valid_acl = pd.read_csv(os.path.join(main_dir, "valid-acl.csv"), header=None, index_col=0).squeeze("columns").to_dict()
valid_meniscus = pd.read_csv(os.path.join(main_dir, "valid-meniscus.csv"), header=None, index_col=0).squeeze("columns").to_dict()

In [ ]:
# Function to resize the depth of a scan to a target depth
def resize_depth(scan, target_depth):
    depth_factor = target_depth / scan.shape[0]
    return zoom(scan, (depth_factor, 1, 1), order=1)

In [ ]:
# Function to pad a scan to a target shape
def pad_to_shape(scan, target_shape):
    padded_scan = np.zeros(target_shape, dtype=scan.dtype)
    min_d, min_h, min_w = min(scan.shape[0], target_shape[0]), min(scan.shape[1], target_shape[1]), min(scan.shape[2], target_shape[2])
    padded_scan[:min_d, :min_h, :min_w] = scan[:min_d, :min_h, :min_w]
    return padded_scan

In [ ]:
# Function to load a specific range of MRI data with labels

def load_mri_data(data_type="train", start_idx=0, end_idx=9, target_shape=(48, 256, 256), target_depth=48):
    """
    Loads MRI data from a specified range and resizes/pads each scan to a target shape.
    Parameters:
    - data_type: "train" or "valid"
    - start_idx, end_idx: Range of file indices to load (e.g., 0 to 9 for train, 1130 to 1249 for valid)
    - target_shape: Target shape for each scan after resizing and padding
    - target_depth: Target depth for each scan to ensure consistent depth
    """
    # Set data path and range
    data_path = train_path if data_type == "train" else valid_path
    axial_path, coronal_path, sagittal_path = Path(data_path) / "axial", Path(data_path) / "coronal", Path(data_path) / "sagittal"

    # Select the appropriate labels dictionary based on data type
    abnormal_labels = train_abnormal if data_type == "train" else valid_abnormal
    acl_labels = train_acl if data_type == "train" else valid_acl
    meniscus_labels = train_meniscus if data_type == "train" else valid_meniscus

    # Initialize lists to store data and labels
    mri_data, labels = [], []

    # Load each MRI scan within the specified range
    for i in range(start_idx, end_idx + 1):
        # Generate file name with zero-padded format (e.g., 0000, 0001, ...)
        file_name = f"{i:04}.npy"

        # Load and process each view with resizing and padding
        axial_scan = pad_to_shape(resize_depth(np.load(axial_path / file_name), target_depth), target_shape)
        coronal_scan = pad_to_shape(resize_depth(np.load(coronal_path / file_name), target_depth), target_shape)
        sagittal_scan = pad_to_shape(resize_depth(np.load(sagittal_path / file_name), target_depth), target_shape)

        # Combine the three views into one structure (3, depth, height, width)
        combined_scan = np.stack([axial_scan, coronal_scan, sagittal_scan], axis=0)
        mri_data.append(combined_scan)

        # Retrieve actual labels for the current scan
        abnormal_label = abnormal_labels.get(i, 0)  # Default to 0 if label is missing
        acl_label = acl_labels.get(i, 0)
        meniscus_label = meniscus_labels.get(i, 0)

        # Append the actual labels
        labels.append({"abnormal": abnormal_label, "acl": acl_label, "meniscus": meniscus_label})

    return np.array(mri_data), labels

In [ ]:
# Define the parameters for batch loading with exact final index coverage
start_indices = list(range(0, 1130, 100))
end_indices = [min(start + 99, 1129) for start in start_indices]  # Ensure final batch ends at 1129

# Function to load a batch of data given start and end indices
def load_batch(start, end):
    return load_mri_data(data_type="train", start_idx=start, end_idx=end)

# Initialize lists to store all data and labels
all_data, all_labels = [], []

# Use ThreadPoolExecutor to parallelize data loading for all batches
with ThreadPoolExecutor() as executor:
    # Launch parallel tasks for loading each batch
    future_to_indices = {executor.submit(load_batch, start, end): (start, end) for start, end in zip(start_indices, end_indices)}

    for future in as_completed(future_to_indices):
        data, labels = future.result()
        all_data.append(data)
        all_labels.extend(labels)  # Extend to add lists of labels directly

# Concatenate all data batches into a single array
train_data = np.concatenate(all_data, axis=0)
train_labels = all_labels  # Already extended to combine all label lists

# Check the final shape of training data and labels
print("Final training data shape:", train_data.shape)  # Expected: (1130, 3, 48, 256, 256)
print("Final number of training labels:", len(train_labels))  # Should match the number of samples in train_data


Final training data shape: (1130, 3, 48, 256, 256)
Final number of training labels: 1130


In [ ]:
# Load the validation data from indices 1130 to 1249
valid_data, valid_labels = load_mri_data(data_type="valid", start_idx=1130, end_idx=1249)

# Check data shapes and labels
print("Validation data shape:", valid_data.shape)  # Expected: (120, 3, 48, 256, 256)

Validation data shape: (120, 3, 48, 256, 256)


In [ ]:
# Select the axial view (index 0)
train_data_axial = train_data[:, 0, :, :, :]  # (num_samples, 48, 256, 256)
valid_data_axial = valid_data[:, 0, :, :, :]  # (num_samples, 48, 256, 256)

# Reshape to 2D projections by averaging along the depth dimension
train_data_reshaped = train_data_axial.mean(axis=1)  # Average along depth for projection
valid_data_reshaped = valid_data_axial.mean(axis=1)

# Normalize to [0, 1] range
train_data_normalized = train_data_reshaped / 255.0
valid_data_normalized = valid_data_reshaped / 255.0

# Final shapes for PyTorch: (batch_size, channels, height, width)
# Add channel dimension as the first axis
train_data_final = torch.tensor(train_data_normalized, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
valid_data_final = torch.tensor(valid_data_normalized, dtype=torch.float32).unsqueeze(1)

# Repeat the channel dimension to create 3 channels
train_data_final = train_data_final.repeat(1, 3, 1, 1)
valid_data_final = valid_data_final.repeat(1, 3, 1, 1)

print("Train data shape:", train_data_final.shape)  # Expected: (1130, 3, 256, 256)
print("Validation data shape:", valid_data_final.shape)  # Expected: (120, 3, 256, 256)

Train data shape: torch.Size([1130, 3, 256, 256])
Validation data shape: torch.Size([120, 3, 256, 256])


In [ ]:
# Transform labels into binary matrices for multi-label classification
# Each label dictionary is converted into a list of binary values for each class
train_labels_matrix = [[lbl['abnormal'], lbl['acl'], lbl['meniscus']] for lbl in train_labels]
valid_labels_matrix = [[lbl['abnormal'], lbl['acl'], lbl['meniscus']] for lbl in valid_labels]

# Convert label matrices to PyTorch tensors
# Using float32 because loss functions like BCEWithLogitsLoss expect float inputs
train_labels_encoded = torch.tensor(train_labels_matrix, dtype=torch.float32, device=device)  # Send directly to device
valid_labels_encoded = torch.tensor(valid_labels_matrix, dtype=torch.float32, device=device)  # Send directly to device

# Verify tensor shapes and ensure the data has been prepared correctly
print("Train labels shape:", train_labels_encoded.shape)  # Expected: (1130, 3)
print("Validation labels shape:", valid_labels_encoded.shape)  # Expected: (120, 3)


Train labels shape: torch.Size([1130, 3])
Validation labels shape: torch.Size([120, 3])


In [ ]:
class MRIModel(nn.Module):
    def __init__(self, num_classes=3):
        super(MRIModel, self).__init__()
        self.backbone = timm.create_model('resnet200d', pretrained=True, num_classes=0)
        self.fc1 = nn.Linear(self.backbone.num_features, 512)  # backbone.num_features should be 2048
        self.fc2 = nn.Linear(512, 256)
        self.fc_out = nn.Linear(256, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc_out(x)
        return self.sigmoid(x)


In [ ]:
train_dataset = TensorDataset(train_data_final, train_labels_encoded)
valid_dataset = TensorDataset(valid_data_final, valid_labels_encoded)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

for data, labels in train_loader:
    print(f"Data batch shape: {data.shape}")  # Expected: (batch_size, 3, 256, 256)
    print(f"Labels batch shape: {labels.shape}")  # Expected: (batch_size, 3)
    break


Data batch shape: torch.Size([16, 3, 256, 256])
Labels batch shape: torch.Size([16, 3])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MRIModel(num_classes=3).to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    for data, labels in loader:
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.size(0)

        # Store predictions and true labels for metrics
        all_preds.append(outputs.detach().cpu())
        all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Compute metrics
    all_preds_binary = (all_preds > 0.5).int()
    accuracy = accuracy_score(all_labels.numpy(), all_preds_binary.numpy())
    precision = precision_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)
    recall = recall_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)

    return running_loss / len(loader.dataset), accuracy, precision, recall



In [ ]:

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data, labels in loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * data.size(0)

            # Store predictions and true labels for metric calculation
            all_preds.append(outputs.cpu())
            all_labels.append(labels.cpu())
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Compute metrics
    all_preds_binary = (all_preds > 0.5).int()
    accuracy = accuracy_score(all_labels.numpy(), all_preds_binary.numpy())
    precision = precision_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)
    recall = recall_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)

    return running_loss / len(loader.dataset), accuracy, precision, recall


In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    # Training
    train_loss, train_accuracy, train_precision, train_recall = train_epoch(
        model, train_loader, criterion, optimizer, device
    )

    # Validation
    val_loss, val_accuracy, val_precision, val_recall = validate_epoch(
        model, valid_loader, criterion, device
    )

    # Print metrics
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}")


Epoch 1/20
Train Loss: 0.5491, Accuracy: 0.3779, Precision: 0.3553, Recall: 0.3401
Validation Loss: 0.6341, Accuracy: 0.1583, Precision: 0.4306, Recall: 0.3590
Epoch 2/20
Train Loss: 0.4359, Accuracy: 0.4292, Precision: 0.6008, Recall: 0.4415
Validation Loss: 0.6245, Accuracy: 0.2250, Precision: 0.4870, Recall: 0.4644
Epoch 3/20
Train Loss: 0.3037, Accuracy: 0.6389, Precision: 0.8191, Recall: 0.6945
Validation Loss: 0.6581, Accuracy: 0.2583, Precision: 0.7466, Recall: 0.6712
Epoch 4/20
Train Loss: 0.1930, Accuracy: 0.7823, Precision: 0.8844, Recall: 0.8451
Validation Loss: 0.6194, Accuracy: 0.3500, Precision: 0.7580, Recall: 0.6391
Epoch 5/20
Train Loss: 0.1308, Accuracy: 0.8425, Precision: 0.9147, Recall: 0.9164
Validation Loss: 0.8839, Accuracy: 0.3667, Precision: 0.7480, Recall: 0.7464
Epoch 6/20
Train Loss: 0.0969, Accuracy: 0.9018, Precision: 0.9525, Recall: 0.9313
Validation Loss: 0.7643, Accuracy: 0.3250, Precision: 0.7335, Recall: 0.6751
Epoch 7/20
Train Loss: 0.0646, Accuracy:

## Explanation of Results of ResNet 200d:

### **Epochs 1-5**

| Epoch | Train Loss | Train Accuracy | Train Precision | Train Recall | Validation Loss | Validation Accuracy | Validation Precision | Validation Recall |
|-------|------------|----------------|------------------|--------------|-----------------|---------------------|----------------------|--------------------|
| 1     | 0.5491     | 0.3779         | 0.3553           | 0.3401       | 0.6341          | 0.1583              | 0.4306               | 0.3590             |
| 2     | 0.4359     | 0.4292         | 0.6008           | 0.4415       | 0.6245          | 0.2250              | 0.4870               | 0.4644             |
| 3     | 0.3037     | 0.6389         | 0.8191           | 0.6945       | 0.6581          | 0.2583              | 0.7466               | 0.6712             |
| 4     | 0.1930     | 0.7823         | 0.8844           | 0.8451       | 0.6194          | 0.3500              | 0.7580               | 0.6391             |
| 5     | 0.1308     | 0.8425         | 0.9147           | 0.9164       | 0.8839          | 0.3667              | 0.7480               | 0.7464             |

**Observations:**
- **Training Metrics:** There's a significant improvement in training loss and accuracy, with precision and recall steadily increasing. By Epoch 5, the model achieves high accuracy (~84%) and strong precision and recall scores (~91% each), indicating effective learning on the training data.
- **Validation Metrics:** Despite improvements in training, validation loss fluctuates and even increases by Epoch 5. Validation accuracy remains substantially lower (~36%) compared to training, and precision and recall are inconsistent. This discrepancy suggests that the model is starting to **overfit** the training data, capturing noise rather than generalizable patterns.

---

### **Epochs 6-10**

| Epoch | Train Loss | Train Accuracy | Train Precision | Train Recall | Validation Loss | Validation Accuracy | Validation Precision | Validation Recall |
|-------|------------|----------------|------------------|--------------|-----------------|---------------------|----------------------|--------------------|
| 6     | 0.0969     | 0.9018         | 0.9525           | 0.9313       | 0.7643          | 0.3250              | 0.7335               | 0.6751             |
| 7     | 0.0646     | 0.9345         | 0.9712           | 0.9538       | 1.0538          | 0.3667              | 0.7835               | 0.6024             |
| 8     | 0.0437     | 0.9611         | 0.9819           | 0.9744       | 0.9267          | 0.3917              | 0.7149               | 0.8131             |
| 9     | 0.0407     | 0.9637         | 0.9844           | 0.9788       | 1.2459          | 0.4000              | 0.7537               | 0.7687             |
| 10    | 0.0380     | 0.9602         | 0.9825           | 0.9789       | 0.9063          | 0.4333              | 0.7704               | 0.7913             |

**Observations:**
- **Training Metrics:** Training continues to excel, with loss dropping below 0.05 and accuracy exceeding 96%. Precision and recall remain exceptionally high (~98% and ~97%, respectively), reinforcing the model's strong fit to the training data.
- **Validation Metrics:** Validation loss spikes significantly in Epochs 7 and 9, while validation accuracy shows marginal improvements, peaking around ~43%. Precision and recall on validation data fluctuate without consistent gains. The widening gap between training and validation performance underscores ongoing **overfitting**, where the model's ability to generalize to unseen data is compromised.

---

### **Epochs 11-15**

| Epoch | Train Loss | Train Accuracy | Train Precision | Train Recall | Validation Loss | Validation Accuracy | Validation Precision | Validation Recall |
|-------|------------|----------------|------------------|--------------|-----------------|---------------------|----------------------|--------------------|
| 11    | 0.0589     | 0.9354         | 0.9713           | 0.9616       | 0.9694          | 0.3250              | 0.7605               | 0.6847             |
| 12    | 0.0324     | 0.9619         | 0.9843           | 0.9723       | 1.1261          | 0.3500              | 0.7513               | 0.6634             |
| 13    | 0.0238     | 0.9779         | 0.9893           | 0.9870       | 0.9199          | 0.3917              | 0.7459               | 0.7699             |
| 14    | 0.0139     | 0.9841         | 0.9942           | 0.9923       | 1.0984          | 0.3167              | 0.7281               | 0.7767             |
| 15    | 0.0221     | 0.9788         | 0.9904           | 0.9896       | 1.0497          | 0.3583              | 0.7866               | 0.6636             |

**Observations:**
- **Training Metrics:** The model continues to improve, reaching up to ~98% accuracy and maintaining near-perfect precision and recall (~99% each). This indicates an even deeper **overfitting**, as the model becomes increasingly specialized to the training data.
- **Validation Metrics:** Validation loss remains high and volatile, with minor fluctuations in accuracy. Precision and recall show some improvement but remain inconsistent. The persistent high validation loss alongside soaring training performance further confirms that the model is **overfitting** and struggling to generalize.

---

### **Epochs 16-20**

| Epoch | Train Loss | Train Accuracy | Train Precision | Train Recall | Validation Loss | Validation Accuracy | Validation Precision | Validation Recall |
|-------|------------|----------------|------------------|--------------|-----------------|---------------------|----------------------|--------------------|
| 16    | 0.0270     | 0.9743         | 0.9863           | 0.9845       | 1.0722          | 0.3250              | 0.7574               | 0.7507             |
| 17    | 0.0246     | 0.9743         | 0.9858           | 0.9859       | 1.1319          | 0.3833              | 0.7845               | 0.6420             |
| 18    | 0.0108     | 0.9903         | 0.9975           | 0.9915       | 1.1616          | 0.3333              | 0.7454               | 0.7514             |
| 19    | 0.0099     | 0.9929         | 0.9944           | 0.9964       | 1.2466          | 0.3000              | 0.7410               | 0.6615             |
| 20    | 0.0142     | 0.9903         | 0.9939           | 0.9927       | 1.1957          | 0.3250              | 0.7485               | 0.6604             |

**Observations:**
- **Training Metrics:** By Epoch 20, training loss plummets to ~0.0099, with accuracy soaring to ~99%. Precision and recall remain extraordinarily high (~99% each). This extreme performance on training data is a clear sign of **overfitting**, as the model has learned the training data almost perfectly.
- **Validation Metrics:** Validation loss continues to increase, and validation accuracy shows no consistent improvement, hovering around ~30-38%. Precision and recall on validation data remain low and unstable. The stark contrast between training and validation performance solidifies the observation that the model is **overfitting** the training dataset.

---

### **Summary of Metrics**

- **Training Performance:**
  - **Loss:** Decreases consistently from ~0.55 to ~0.0099.
  - **Accuracy:** Rises sharply from ~38% to ~99%.
  - **Precision & Recall:** Both metrics soar, indicating that the model is making highly accurate and comprehensive predictions on the training data.

- **Validation Performance:**
  - **Loss:** Generally increases from ~0.63 to ~1.25, with minor fluctuations.
  - **Accuracy:** Remains low (~15-43%) and does not follow the training trend.
  - **Precision & Recall:** Fluctuate without significant improvement, indicating inconsistent performance on unseen data.

**Conclusion:** The ResNet-200d model exhibits clear **overfitting**. While it performs exceptionally well on the training data, its ability to generalize to validation data is poor, as evidenced by the increasing validation loss and stagnant validation accuracy.

**Next Steps:** To address the overfitting issue, we will now try a **3D ResNet**. This architecture is better suited for volumetric data like MRI scans, potentially enhancing the model's ability to capture spatial hierarchies and improving generalization performance.